In [ ]:
# Reading in the macthed cell barcodes and guides that were generated in wynton
guide_cell_barcodes = pd.read_csv('/Users/brendamelano/Desktop/Reprogramming_Osteosarcoma/Perturb_seq_analysis/OS384/OS384_guide_cell_barcode_matches_counts.txt', delimiter='\t', index_col=0, header=None) 


# Adding the '-1' string to the guide cell barcodes read in
guide_cell_barcodes.index = [str(idx) + '-1' for idx in guide_cell_barcodes.index]


# Shifting the index to an actual column
guide_cell_barcodes = guide_cell_barcodes.reset_index()


# Renaming the column names
guide_cell_barcodes = guide_cell_barcodes.rename(columns={guide_cell_barcodes.columns[0]: 'cell_barcode', 
                                                          guide_cell_barcodes.columns[1]: 'guide',
                                                          guide_cell_barcodes.columns[2]: 'count'})


# Keeping the cell_barcode/guide pair that has the highest count to move forward with the rest of the analysis
result_df = guide_cell_barcodes.loc[guide_cell_barcodes.groupby('cell_barcode')['count'].idxmax()].reset_index(drop=True)


# Creating a new column in the adata.obs object for the actual cell barcode
adata.obs = adata.obs.assign(cell_barcode=adata.obs.index)


# Creating a dataframe for the adata.obs object
adata_obs_df = adata.obs


# Perform an inner join on the 'cell_barcode' column
matched_df = adata_obs_df.merge(result_df, on='cell_barcode', how='inner')


# Now matched_df contains only the rows where the cell_barcode values matched in both DataFrames


matched_df_unique = matched_df.drop_duplicates(subset=['cell_barcode'], keep='first')


# Reading in the guide key
Gene_guide_key = pd.read_csv('/Users/brendamelano/Desktop/Reprogramming_Osteosarcoma/Perturb_seq_analysis/Perturb_osteo_whitelist_guides .csv')


Gene_guide_key = Gene_guide_key.iloc[:, [1, 2]]


# Rename the columns
Gene_guide_key.columns = ['guide', 'Gene']


matched_df_unique = matched_df_unique.merge(Gene_guide_key, on='guide', how='left')


# Dropping the guide column
matched_df_unique = matched_df_unique.drop('guide', axis=1)


matched_df_unique = matched_df_unique.drop('count', axis=1)


matched_df_unique = matched_df_unique.drop_duplicates()


matched_df_unique = matched_df_unique.set_index('cell_barcode')


for barcode in matched_df_unique.index:
    if barcode in adata.obs.index:
        # If barcode exists in AnnData, add/update gene info in .obs
        # Example: Add a concatenated string of genes to a new column, handling duplicates
        genes = matched_df_unique.loc[barcode, 'Gene']
        gene_str = ', '.join(genes) if isinstance(genes, pd.Series) else genes
        adata.obs.at[barcode, 'Gene'] = gene_str
    else:
        # If barcode does not exist, add a new entry (optional, depends on your analysis needs)
        # You might need to handle adding new cells to `.X` or other attributes depending on your data structure
        genes = 'control'
        
adata.obs['Gene'] = adata.obs['Gene'].fillna('control')

adata.var_names_make_unique()